In [ ]:
# Whisperで日本語音声を認識しLRCファイルを生成する
#このノートブックはGoogle Colab上でGPUを使って日本語音声を処理し、
# タイムスタンプ付きのLRC歌詞ファイルを生成するためのものです。

In [ ]:
# 1. 必要なライブラリをインストール
!pip install -q openai-whisper
!pip install -q torch torchvision torchaudio

# GPUが利用可能か確認
import torch
print(f"CUDA 利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU デバイス: {torch.cuda.get_device_name(0)}")


In [ ]:
# 2. 必要なライブラリをインポート
import whisper
import os
from datetime import timedelta

def format_timestamp(seconds):
    """秒数をLRC形式のタイムスタンプ [mm:ss.xx] に変換"""
    td = timedelta(seconds=seconds)
    total_seconds = int(td.total_seconds())
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    centiseconds = int((td.total_seconds() - total_seconds) * 100)
    return f"[{minutes:02d}:{seconds:02d}.{centiseconds:02d}]"

def generate_lrc(segments, output_path):
    """Whisperの分割結果からLRCファイルを生成"""
    with open(output_path, 'w', encoding='utf-8') as f:
        # LRCファイルのヘッダーを書き込む
        f.write("[ar:Unknown]\n")
        f.write("[ti:Unknown]\n")
        f.write("[by:Whisper AI]\n")
        f.write("[00:00.00]\n")
        
        # 各行の歌詞とタイムスタンプを書き込む
        for segment in segments:
            timestamp = format_timestamp(segment['start'])
            text = segment['text'].strip()
            f.write(f"{timestamp}{text}\n")
    
    print(f"LRCファイルが生成されました: {output_path}")

print("関数定義完了!")


In [ ]:
# 3. Whisperモデルを読み込む
# 選択可能なモデル: tiny / base / small / medium / large
# 日本語認識には medium または large モデルを推奨（高精度）
# GPUメモリが不足する場合は small か base を選択

model_size = "medium"  # 必要に応じて変更
print(f"Whisper {model_size} モデルを読み込んでいます...")
model = whisper.load_model(model_size)
print(f"モデルの読み込みが完了しました! 使用デバイス: {model.device}")


In [ ]:
# 4. 音声ファイルをアップロード（Colab想定）
from google.colab import files

print("音声ファイルをアップロードしてください...")
uploaded = files.upload()

# アップロードしたファイル名を取得
audio_file = list(uploaded.keys())[0]
print(f"アップロード済みファイル: {audio_file}")


In [ ]:
# 5. Whisperで日本語音声を認識
print("音声の認識を開始します...")
print("これには数分かかる場合があります。音声の長さとモデルのサイズによります...")

# GPUで転写し、言語を日本語に指定
result = model.transcribe(
    audio_file,
    language="ja",  # 言語に日本語を指定
    verbose=True,   # 進捗を表示
    task="transcribe"  # 転写タスク
)

print("\n認識完了!")
print(f"検出された言語: {result['language']}")
print(f"テキスト: {result['text'][:100]}...")  # 先頭100文字を表示


In [ ]:
# 6. LRCファイルを生成
# 出力ファイル名を設定
lrc_filename = os.path.splitext(audio_file)[0] + ".lrc"

# LRCファイルを作成
generate_lrc(result['segments'], lrc_filename)

# 先頭数行をプレビュー
print("\nLRCファイルのプレビュー:")
with open(lrc_filename, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines[:10]:  # 先頭10行を表示
        print(line.rstrip())


In [ ]:
# 7. LRCファイルをダウンロード（Colab想定）
from google.colab import files

print(f"LRCファイルをダウンロード: {lrc_filename}")
files.download(lrc_filename)


## オプション：複数の音声ファイルを一括処理

複数の音声ファイルを処理する必要がある場合は、以下のコードを使用できます：


In [ ]:
# バッチ処理用の関数
import glob

def process_audio_to_lrc(audio_path, model, output_dir="./"):
    """
    単一の音声ファイルを処理してLRCファイルを生成
    
    パラメータ:
        audio_path: 音声ファイルのパス
        model: 読み込み済みのWhisperモデル
        output_dir: 出力ディレクトリ
    """
    print(f"\nファイルを処理中: {audio_path}")
    
    try:
        # 音声を認識
        result = model.transcribe(
            audio_path,
            language="ja",
            verbose=False,
            task="transcribe"
        )
        
        # LRCファイルを作成
        filename = os.path.basename(audio_path)
        lrc_filename = os.path.splitext(filename)[0] + ".lrc"
        lrc_path = os.path.join(output_dir, lrc_filename)
        
        generate_lrc(result['segments'], lrc_path)
        print(f"✓ 完了: {lrc_filename}")
        return lrc_path
    except Exception as e:
        print(f"✗ エラー: {audio_path} - {str(e)}")
        return None

def batch_process_folder(folder_path, model, output_dir=None):
    """
    指定したフォルダ内のすべての音声ファイルを一括処理
    
    パラメータ:
        folder_path: 音声ファイルが含まれるフォルダのパス
        model: 読み込み済みのWhisperモデル
        output_dir: 出力ディレクトリ（Noneの場合は元のフォルダに出力）
    """
    # 出力フォルダが未指定なら入力フォルダを使う
    if output_dir is None:
        output_dir = folder_path
    
    # 出力フォルダが無ければ作成
    os.makedirs(output_dir, exist_ok=True)
    
    # 対応する音声フォーマット
    audio_extensions = ['*.mp3', '*.wav', '*.m4a', '*.flac', '*.ogg', '*.aac', '*.wma']
    
    # すべての音声ファイルを取得
    audio_files = []
    for ext in audio_extensions:
        pattern = os.path.join(folder_path, ext)
        audio_files.extend(glob.glob(pattern))
        # 拡張子の大文字も検索
        pattern_upper = os.path.join(folder_path, ext.upper())
        audio_files.extend(glob.glob(pattern_upper))
    
    if not audio_files:
        print(f"フォルダ '{folder_path}' 内に音声ファイルが見つかりませんでした")
        return []
    
    print(f"{len(audio_files)} 個の音声ファイルが見つかりました")
    print("=" * 50)
    
    # 各ファイルを処理
    results = []
    for i, audio_file in enumerate(audio_files, 1):
        print(f"\n[{i}/{len(audio_files)}] 処理中...")
        lrc_path = process_audio_to_lrc(audio_file, model, output_dir)
        if lrc_path:
            results.append(lrc_path)
    
    print("\n" + "=" * 50)
    print(f"一括処理完了! 成功: {len(results)}/{len(audio_files)}")
    return results

print("一括処理関数が定義されました!")


In [ ]:
# 8. 一括処理: 音声を含むZIPをアップロード
from google.colab import files
import zipfile

print("音声ファイルを含むZIP圧縮ファイルをアップロードしてください...")
uploaded_zip = files.upload()

if uploaded_zip:
    # アップロードしたZIPのファイル名を取得
    zip_filename = list(uploaded_zip.keys())[0]
    
    # 展開先ディレクトリを作成
    extract_dir = "audio_files"
    os.makedirs(extract_dir, exist_ok=True)
    
    # ファイルを解凍
    print(f"\n{zip_filename} を解凍しています...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    
    print(f"ファイルは次の場所に解凍されました: {extract_dir}")
    
    # 展開したファイルを表示
    print("\n解凍されたファイル:")
    for root, dirs, files in os.walk(extract_dir):
        for file in files:
            print(f"  - {os.path.join(root, file)}")


In [ ]:
# 9. バッチ処理を実行
# 処理対象のフォルダパスを指定
input_folder = "audio_files"  # 自分のフォルダパスに変更
output_folder = "lrc_output"  # LRCファイルの出力先

# バッチ処理を開始
print(f"フォルダの一括処理を開始: {input_folder}")
print(f"LRCファイルの保存先: {output_folder}\n")

lrc_files = batch_process_folder(input_folder, model, output_folder)

print(f"\n生成されたLRCファイル:")
for lrc_file in lrc_files:
    print(f"  - {lrc_file}")


In [ ]:
# 10. 生成したLRCファイルをZIPにまとめてダウンロード
import shutil

# ZIPファイルを作成
output_zip = "lrc_files.zip"
print(f"LRCファイルをパッケージングしています...")

shutil.make_archive(
    output_zip.replace('.zip', ''),  # 拡張子なしのファイル名
    'zip',                            # 圧縮形式
    output_folder                     # 圧縮対象のフォルダ
)

print(f"パッケージング完了: {output_zip}")

# ZIPファイルをダウンロード
from google.colab import files
files.download(output_zip)


## 使用説明

### 単一ファイルの処理フロー
1. **依存関係をインストール**：1番目のコードセルを実行して必要なライブラリをインストール
2. **ライブラリをインポートして関数を定義**：2番目のコードセルを実行
3. **モデルを読み込む**：3番目のコードセルを実行（モデルサイズを変更可能）
   - `tiny`: 最速、精度は低め
   - `base`: 高速、精度は普通
   - `small`: 速度と精度のバランス
   - `medium`: やや遅い、精度高い（日本語推奨）
   - `large`: 最も遅い、最も精度が高い
4. **音声をアップロード**：4番目のコードセルを実行して音声ファイルをアップロード
5. **音声を認識**：5番目のコードセルを実行して認識開始
6. **LRCを生成**：6番目のコードセルを実行してLRCファイルを生成
7. **ファイルをダウンロード**：7番目のコードセルを実行して生成されたLRCファイルをダウンロード

### バッチ処理フロー（推奨）
1. **依存関係をインストール**：1番目のコードセルを実行
2. **ライブラリをインポートして関数を定義**：2番目のコードセルを実行
3. **モデルを読み込む**：3番目のコードセルを実行
4. **一括処理関数を定義**：9番目のコードセル（「オプション：一括処理」セクション）を実行
5. **圧縮ファイルをアップロード**：10番目のコードセルを実行し、すべての音声ファイルを含むZIP圧縮ファイルをアップロード
6. **一括処理**：11番目のコードセルを実行し、フォルダ内のすべての音声ファイルを自動処理
7. **結果をダウンロード**：12番目のコードセルを実行し、パッケージ化されたすべてのLRCファイルをダウンロード

**対応する音声フォーマット**: MP3, WAV, M4A, FLAC, OGG, AAC, WMA など

**一括処理の利点**：
- フォルダ内のすべての音声ファイルを自動認識
- 複数の音声フォーマットに対応
- 自動的に出力ディレクトリを作成
- 処理進捗と成功率を表示
- エラー処理機能付き、1つのファイルが失敗しても他のファイルに影響なし
- 生成されたすべてのLRCファイルをパッケージ化してダウンロード可能
